In [ ]:
#Set up paths
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF

# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime
import yaml
import glob
#from box import Box #???

importlib.reload( LL )
importlib.reload( uti )
importlib.reload( nuti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(GrU)

# Initialize regridding object library to empty dict 
RgOb=GrU.regrid_object_lib(initialize=True)


rgob = GrU.regrid_object_lib(RgOb=RgOb,src='ne240pg3', dst='fv1x1'   ) #, RegridMethod='CONSERVE_2ND',initialize=False)
rgob = GrU.regrid_object_lib(RgOb=RgOb,src='ne30pg3', dst='fv1x1'   ) #, RegridMethod='CONSERVE_2ND',initialize=False)



In [ ]:
grid='fv0.9x1.25'

dir1=f'/glade/campaign/cgd/amp/juliob/ERA5/{grid}/L93/'

ihr=6
iday=20
imo=6
iyr=2004
ird=0

for iday in np.arange( start=1,stop=31 ):
    if imo==6 and iday==1:
        start=6
    else:
        start=0
    for ihr in np.arange( start=start, stop=24, step=6 ):
    
        iss=ihr*3600
        
        ymds=f'{str(iyr).zfill(4)}-{str(imo).zfill(2)}-{str(iday).zfill(2)}-{str(iss).zfill(5)}'
        
        file1=f'ERA5_x_{grid}_L93_rgC1_WO.{ymds}.nc'
        print( file1 )
        X1=xr.open_dataset( dir1+file1 )
        
        u_era = X1.U.values
        
        
        
        A = uti.MakeDict4Exp( exp="c153_topfix_ne240pg3_FMTHIST_xic_x02", user='juliob', subd='hist' , 
                             hsPat='cam.h1i' , ymdPat=ymds,verbose=True, open_dataset=True )
        
        B = uti.MakeDict4Exp( exp="c153_ne30pg3_FMTHIST_x05", user='juliob', subd='hist' , 
                             hsPat='cam.h1i' , ymdPat=ymds,verbose=True, open_dataset=True )
        
        u_cQ=A.X.U.values
        u_c1=B.X.U.values
        
        rgobX = GrU.regrid_object_lib(RgOb=RgOb,src='ne240pg3', dst='fv1x1'   )
        u_cQ_x1 =RgF.Horz(xfld_Src=u_cQ, Src='ne240pg3', Dst='fv1x1' , RegridObj_In=  rgobX  ) 
        
        rgobX = GrU.regrid_object_lib(RgOb=RgOb,src='ne30pg3', dst='fv1x1'   )
        u_c1_x1 =RgF.Horz(xfld_Src=u_c1, Src='ne30pg3', Dst='fv1x1' , RegridObj_In=  rgobX  ) 
        
        d_c1 = u_c1_x1 - u_era
        d_cQ = u_cQ_x1 - u_era
        
        ad_c1 = np.sqrt(np.average( d_c1**2 , axis=3 ))
        ad_cQ = np.sqrt(np.average( d_cQ**2 , axis=3 ))
        
        if ird == 0:
            zlev=-7*np.log( A.X.lev.values/1_000. )
            lat=X1.lat.values
            ad_c1_t = np.tile( ad_c1, (4*31,1,1) )
            ad_cQ_t = np.tile( ad_cQ, (4*31,1,1) )
            print(np.shape(ad_c1_t))

        ad_c1_t[ird,:,:] = ad_c1[0,:,:] 
        ad_cQ_t[ird,:,:] = ad_cQ[0,:,:]
        ird=ird+1


In [ ]:
print(ad_cQ_t.shape)
hours=np.arange(124)/4.


In [ ]:
dolev = np.linspace(-1,1,num=21)
olev = np.linspace(-150,150,num=51)
# Create a figure with a 2x2 grid of subplots
fig, ax = plt.subplots(1, 3, figsize=(30, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()


k=80
co=ax[0].contourf(  u_cQ_x1[0,k,:,:] , levels = olev, cmap='bwr' )
cb=plt.colorbar( co )
co=ax[1].contourf(  u_c1_x1[0,k,:,:]  , levels = olev , cmap='bwr' )
cb=plt.colorbar( co )
co=ax[2].contourf(  u_era[0,k,:,:]  , levels = olev , cmap='bwr' )
cb=plt.colorbar( co )
#co=ax[2].contourf( u_x1[0,k,:,:]-u_era[0,k,:,:]  , levels = olev , cmap='bwr' )
#cb=plt.colorbar( co )

In [ ]:
plt.contourf( d_c1[0,:,37,:] , levels=olev/5, cmap='bwr' )

In [ ]:
y1=np.average(ad_c1_t[:,:,27:47],axis=2)
yQ=np.average(ad_cQ_t[:,:,27:47],axis=2)



In [ ]:
plt.plot( hours, y1[:,51] )
plt.plot( hours, yQ[:,51] )

In [ ]:

fig, ax = plt.subplots(1, 3, figsize=(30, 5))

# Flatten axs to easily access each subplot by index
ax = ax.flatten()

y0,y1=20,50 #27,57 #125,145


co=ax[0].contourf(hours,zlev, np.average(ad_c1_t[:,:,y0:y1],axis=2).T, levels=np.linspace(0,40,num=41))
plt.colorbar(co)
co=ax[1].contourf(hours,zlev,np.average(ad_cQ_t[:,:,y0:y1],axis=2).T , levels=np.linspace(0,40,num=41))
plt.colorbar(co)

In [ ]:
print(lat[27])